# Are We There Yet?

You are visiting your friend and it will require
* A 5 minute walk to the ‘L’ station
* Waiting for the train, which arrives every 20 minutes
* Traveling 30 minutes by ‘L’
* Catching a taxi at the ‘L’ destination
  * There is a 20% chance that the car is waiting for you
  * Otherwise the average wait time is 10 minutes
* A 12 minute taxi ride 

_How long should you plan for the trip to take?_

#### Some LaTeX macros (hidden in math mode, will not render correctly in VS Code)
$
\newcommand{\vh}{\boldsymbol{h}}
\newcommand{\vt}{\boldsymbol{t}}
\newcommand{\vx}{\boldsymbol{x}}
\newcommand{\vX}{\boldsymbol{X}}
\newcommand{\cf}{\mathcal{F}}
\newcommand{\cu}{\mathcal{U}}
\newcommand{\dif}{\mathrm{d}}
\newcommand{\Ex}{\mathbb{E}}
\newcommand{\Prob}{\mathbb{P}}
\newcommand{\disc}{\operatorname{disc}}
\newcommand{\norm}[2]{{\left \lVert #1 \right \rVert}_{#2}}
$

#### Run this notebook in Juypter with the `conda qmcpy` environment or [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QMCSoftware/MATH565Fall2025/blob/main/notebooks/AreWeThereYet.ipynb?flush_cache=true)

In [ ]:
if "google.colab" in str(get_ipython()):
    print("⚡Note: Setup here can take a few minutes.")
    !apt-get -y install cm-super dvipng texlive-latex-extra texlive-latex-recommended
    !pip install git+https://github.com/QMCSoftware/QMCSoftware.git@develop
    !git clone https://github.com/QMCSoftware/MATH565Fall2025.git
    %cd MATH565Fall2025/notebooks
    print("\n✅ Colab setup complete.")
    print("   For faster performance, please install the qmcpy environment locally.")

## Mathematical set-up
$$
\begin{aligned}
T & = \text{travel time} = 5 + W_1 + 30 + W_2 + 12 = 47 + W_1 + W_2 \\
W_1 & = \text{waiting time for 'L' train} \sim \mathcal{U}[0,20] \\
W_2 & = \text{waiting time for taxi}, \quad 
\Prob(W_2 \le w) = \begin{cases} 0, & w < 0 \\ 0.2+ 0.04 \exp(-w/20), & w \ge 0 \end{cases}
\end{aligned}
$$

We assume that $W_1$ and $W_2$ are independent

We will now perform Monte Carlo simulation


## Monte Carlo estimation of the average waiting time

### Import needed packages and initialize some parameters

In [ ]:
import matplotlib.pyplot as plt
import math, time, timeit, sys
import numpy as np
import scipy as sp
from scipy import stats
import qmcpy as qp
import sympy as sy
from IPython.display import Markdown
import mc_class as init
init.init()                 # apply rcParams + numpy error policy
colors = init.get_py_colors()

### Set up random variables $W_1$ and $W_2$

In [ ]:
# --- Parameters --- We define these with variables so that we can change their values later
walk_time = 5  #time to walk to the 'L'
on_L_time = 30  #time on the 'L'
taxi_time = 12  #time in the taxi
T0 = walk_time + on_L_time + taxi_time # fixed time
wait_L_time = 20  #max time until next 'L' train
taxi_now = 0.2  #probability of taxi now
taxi_wait_avg = 10  #average time to wait for a taxi if it is not there


# --- W1: Uniform(0, L_time) ---
W1 = stats.uniform(loc=0, scale=wait_L_time)  #this is the waiting time for the 'L'

# --- W2: Zero-inflated Exponential ---
# Since scipy does not have a zero-inflated exponential we define it with the help of ChatGPT
class zero_inflated_expon(stats.rv_continuous):
    def __init__(self, p_zero=0.2, rate=1.0, *args, **kwargs):
        super().__init__(a=0.0, name="zero_inflated_expon", *args, **kwargs)
        if not (0.0 <= p_zero < 1.0):
            raise ValueError("p_zero must be in [0,1).")
        if rate <= 0:
            raise ValueError("rate must be > 0.")
        self.p_zero = float(p_zero)
        self.rate = float(rate)

    # CDF for completeness (unchanged)
    def _cdf(self, x):
        p0, lam = self.p_zero, self.rate
        x = np.asarray(x)
        return np.where(
            x < 0, 0.0,
            np.where(
                x == 0, p0,
                p0 + (1 - p0) * (1 - np.exp(-lam * x))
            )
        )

    # **Fast closed-form PPF**
    def _ppf(self, q):
        p0, lam = self.p_zero, self.rate
        q = np.asarray(q)
        # map q <= p0 to 0
        res = np.zeros_like(q, dtype=float)
        # handle q in (p0,1)
        mask = (q > p0) & (q < 1.0)
        t = (q[mask] - p0) / (1.0 - p0)  # in (0,1)
        res[mask] = -np.log1p(-t) / lam  # stable when q ~ 1
        # q == 1 -> inf
        res = np.where(q >= 1.0, np.inf, res)
        # q < 0 -> nan (SciPy typically returns nan out-of-domain)
        res = np.where(q < 0.0, np.nan, res)
        return res

    # Optional: vectorized RNG (yours was fine)
    def _rvs(self, size=None, random_state=None):
        p0, lam = self.p_zero, self.rate
        u = random_state.random(size)
        x = np.zeros(size)
        mask = u >= p0
        x[mask] = random_state.exponential(scale=1/lam, size=np.sum(mask))
        return x

# Set the parameters for our 
W2 = zero_inflated_expon(p_zero=taxi_now, rate=1/taxi_wait_avg)

### Compute the true (population) mean travel time

In [ ]:
true_mean_T = T0 + W1.mean() + W2.mean()  #use the mean properties of the random variables, in this case the answer is simple
print(f"Mean travel time = \u03bc ={true_mean_T : .1f}")

### Generate random samples and compute the sample mean travel time

Here we generate

In [ ]:
display(Markdown(fr"""
\begin{{gather}}
W_{{11}}, W_{{21}}, \ldots \overset{{\text{{IID}}}}{{\sim}} \mathcal{{U}}[0,{wait_L_time}] \\
W_{{12}}, W_{{22}}, \ldots \overset{{\text{{IID}}}}{{\sim}} \text{{zero inflated exponential}} \\
T_{{i}} = {T0} + W_{{i1}} + W_{{i2}}, \quad i=1,2,\ldots
\end{{gather}}
"""))

and then compute the sample mean

$$
\widehat{{\mu}}_n = \frac 1n \sum_{{i=1}}^n T_i,\quad n=1,2,\ldots
$$

In [ ]:
n_max = 10**4  #sample size
nrange = np.arange(1, n_max+1)
W1_samples =  W1.rvs(size=n_max)  #generate samples of waiting times for the 'L'
W2_samples =  W2.rvs(size=n_max)  #generate samples of waiting times for the taxi
print("'L' waiting time W1 samples = ",W1_samples[:20])
print("Taxi waiting time W2 samples = ",W2_samples[:20])
T_samples = 47 + W1_samples + W2_samples  #generate samples of total travel time
sample_mean_T = np.cumsum(T_samples) / nrange  #compute the travel time sample means for first 1, 2, ..., n samples

plt.semilogx(sample_mean_T, label="Cumulative sample mean")
plt.axhline(true_mean_T, color=colors["orange"], linestyle='--', label="True mean")
plt.xlabel(r"Number of samples, $n$")
plt.ylabel(r"Sample mean, $\widehat{\mu}_n$")
plt.legend();  #add a semi-colon to suppress output saying that there was a plot

#### Do you notice that
* The sample mean converges to the true (population) mean
* The convergence is not monotonic, but oscillatory
* Each time that you run the cell above the output changes

## What is the convergence rate?

Now let's repeat this experiment $R$ times and look at the average convergence. Notice that the error tends to zero at a rate of $\mathcal{O}(n^{-1/2})$.  For $r =  1, 2, \ldots, R$

In [ ]:
display(Markdown(fr"""
\begin{{gather}}
W_{{11}}^{{(r)}}, W^{{(r)}}_{{21}}, \ldots \overset{{\text{{IID}}}}{{\sim}} \mathcal{{U}}[0,{wait_L_time}] \\
W_{{12}}^{{(r)}}, W_{{22}}^{{(r)}}, \ldots \overset{{\text{{IID}}}}{{\sim}} \text{{zero inflated exponential}} \\
T_{{i}}^{{(r)}} = {T0} + W_{{i1}}^{{(r)}} + W_{{i2}}^{{(r)}}. \quad i=1, 2 \ldots 
\end{{gather}}
"""))

and then compute the sample mean
$$
\widehat{\mu}_n^{(r)} = \frac 1n \sum_{i=1}^n T_i^{(r)}, \quad n = 1, 2, \ldots
$$

In [ ]:
R = 100 #number of replications
W1_R_samples = W1.rvs(size=(n_max,R))  #generate R replications of samples of waiting times for the 'L'
W2_R_samples = W2.rvs(size=(n_max,R))  #generate R replications of samples of waiting times for the taxi
T_R_samples = 47 + W1_R_samples + W2_R_samples  #generate N replications of samples of total travel time
sample_mean_T_R_reps = np.cumsum(T_R_samples, axis = 0) / nrange[:, None]   #compute the travel time sample means for all N replications
error_sample_mean_T = np.abs(true_mean_T - sample_mean_T_R_reps)  #errors
bot_per = 25
top_per = 75
err_bot = np.percentile(error_sample_mean_T, bot_per, axis=1)
err_med = np.percentile(error_sample_mean_T, 50, axis=1)
err_top = np.percentile(error_sample_mean_T, top_per, axis=1)
fig, ax = plt.subplots()
ax.loglog(nrange,err_med, '-', color = colors["blue"], label="median error")
ax.fill_between(nrange, err_bot, err_top, color = colors["blue"],alpha = 0.3, linewidth = 0)
init.plot_rate_line(ax, nrange[[-1,0]], y_start=err_med[-1], rate=0.5, color=colors["orange"])
ax.set_xlabel(r"Number of samples, $n$")
ax.set_ylabel(r"Error, $| \mu - \hat{\mu}^{(r)}_n |$")
ax.set_title(f"$R = {R}$ Replications")
plt.legend(frameon=False);

### Root mean squared error

We know that the sample mean, $\widehat{\mu}_n$ has variance $\sigma^2/n$, where $\sigma^2$ is the variance of $T$.  Since $T$ is a sum of independent random variables, we can compute its variance

In [ ]:
true_var_T = W1.var() + W2.var()  #the variance of a sum of independent variables is the sum of the variances
true_std_T = np.sqrt(true_var_T)
print(f"Variance of travel time = \u03C3\u00B2 ={true_var_T : .1f}, which means standard deviation = \u03C3 ={true_std_T : .1f}")

### Central Limit Theorem

For large $n$, $\widehat{\mu}_n$ is approximately distributed $\mathcal{N}(\mu,\sigma^2/n)$ and so for approximately 
* 25% of the replications, the absolute error would be below the 62.5% percentile of the standard normal times $\sigma/\sqrt{n}$
* 50% of the replications, the absolute error would be below the 75% percentile of the standard normal times $\sigma/\sqrt{n}$
* 75% of the replications, the absolute error would be below the 87.5% percentile of the standard normal times $\sigma/\sqrt{n}$

In [ ]:
fig, ax = plt.subplots()
ax.loglog(nrange,err_med, '-', color = colors["blue"], label="median error")
ax.fill_between(nrange, err_bot, err_top, color = colors["blue"],alpha = 0.3, linewidth = 0)
init.plot_rate_line(ax, nrange[[0,-1]], y_start=true_std_T*stats.norm.ppf(0.75), rate=0.5, color=colors["orange"])
init.plot_rate_line(ax, nrange[[0,-1]], y_start=true_std_T*stats.norm.ppf(0.625), rate=0.5, color=colors["green"], label="")
init.plot_rate_line(ax, nrange[[0,-1]], y_start=true_std_T*stats.norm.ppf(0.875), rate=0.5, color=colors["green"], label="")
ax.set_xlabel(r"Number of samples, $n$")
ax.set_ylabel(r"Error, $| \mu - \hat{\mu}^{(r)}_n |$")
ax.set_title(f"$R = {R}$ Replications")
plt.legend(frameon=False);

## What if we do not know the standard deviation
For realistic examples where we do not know the standard deviation, we can construct an approximate confidence interval



In [ ]:
#First we estimate the standard deviation
confid_level = 0.99
q_confid = - stats.norm.ppf((1-confid_level)/2.)
n_sig = 100  #pilot sample size
W1_sig_samples =  W1.rvs(size=n_sig)  #generate samples of waiting times for the 'L'
W2_sig_samples =  W2.rvs(size=n_sig)  #generate samples of waiting times for the taxi
T_sig_samples = 47 + W1_sig_samples + W2_sig_samples  #generate samples of total travel time
sample_std_T = np.std(T_sig_samples)

#Next we estimate the mean with an independent set of samples
n_mean = 10000  #pilot sample size
W1_mean_samples =  W1.rvs(size=n_mean)  #generate samples of waiting times for the 'L'
W2_mean_samples =  W2.rvs(size=n_mean)  #generate samples of waiting times for the taxi
T_mean_samples = 47 + W1_mean_samples + W2_mean_samples  #generate samples of total travel time
sample_mean_T = np.mean(T_mean_samples)  #compute the travel time sample means for first 1, 2, ..., n samples
confid_interval = sample_mean_T + np.array([-1,1])*(q_confid*sample_std_T)/np.sqrt(n_mean)
print(f"The mean travel time is within the interval [{confid_interval[0]:.1f},{confid_interval[1]:.1f}] with probability about{100*confid_level:.1f}%")

## What about percentiles or quantiles
You may be more interested in the 90% of the travel time, to reduce the chance of arriving late.  This can be calculated analytically for this simple case, but we can also illustrate Monte Carlo using the sample quantile function.

In [ ]:
prob = 0.9  #the quantile value we are interested in
sample_quantiles_T_R_reps = np.quantile(T_R_samples, prob, axis=0)
mean_sample_quantiles = sample_quantiles_T_R_reps.mean()
print(f"With probability{prob*100 : 0.1f}% you will make it to your friend's house within {mean_sample_quantiles : 0.1f} minutes")

## Another kind of sampling

The examples above use independent and identically distributed (IID) sampling.  There is another kind of sampling called low discrepancy (LD) sampling that give faster convergence.  Here is an example.

In [ ]:
n_LD_max = 2**13  #must be a power of 2
n_LD_range = np.arange(1, n_LD_max+1)
LD_X_R_samples = qp.Sobol(dimension=2,replications=R).gen_samples(n_LD_max)
W1_LD_R_samples = W1.ppf(LD_X_R_samples[:,:,0])  #generate R replications of samples of waiting times for the 'L'
W2_LD_R_samples = W2.ppf(LD_X_R_samples[:,:,1])  #generate R replications of samples of waiting times for the taxi
T_LD_R_samples = 47 + W1_LD_R_samples + W2_LD_R_samples  #generate N replications of samples of total travel time
sample_mean_T_LD_R_reps = np.cumsum(T_LD_R_samples, axis = 1) / n_LD_range   #compute the travel time sample means for all N replications
error_sample_mean_T_LD = np.abs(true_mean_T - sample_mean_T_LD_R_reps)  #errors
err_LD_bot = np.percentile(error_sample_mean_T_LD, bot_per, axis=0)
err_LD_med = np.percentile(error_sample_mean_T_LD, 50, axis=0)
err_LD_top = np.percentile(error_sample_mean_T_LD, top_per, axis=0)
fig, ax = plt.subplots()
ax.loglog(nrange,err_med, '-', color = colors["blue"], label="median IID error")
ax.loglog(n_LD_range,err_LD_med, '-', color = colors["purple"], label="median LD error")
ax.fill_between(n_LD_range, err_LD_bot, err_LD_top, color = colors["purple"],alpha = 0.3, linewidth = 0)
ax.set_xlabel(r"Number of samples, $n$")
ax.set_ylabel(r"Error, $| \mu - \hat{\mu}^{(r)}_n |$")
ax.set_title(f"$R = {R}$ Replications")
plt.legend(frameon=False);

#### Notice the faster convergence. This is a teaser for later in the course.

We can begin to see why by plotting IID and LD wait times.  Note that LD has fewer gaps and clusters

In [ ]:
#First the IID sampling
plt.semilogy(W1_samples[:1000],W2_samples[:1000],'.')
plt.xlabel("Waiting for the 'L'")
plt.ylabel("Waiting for the taxi")
plt.title("IID Sampling");

In [ ]:
#Next the LD sampling
plt.semilogy(W1_LD_R_samples[0,:1000],W2_LD_R_samples[0,:1000],'.')
plt.xlabel("Waiting for the 'L'")
plt.ylabel("Waiting for the taxi")
plt.title("LD Sampling");